In [1]:
import psycopg2
from config import local_host, local_name, local_port, local_user
from creds import local_db_password
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time

from sim_v1 import simulate

warnings.simplefilter('ignore')
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()
season = 2023

In [2]:
def check_roll_params(setup=None, param_name='rolling_params'):
    total = 0
    for key in setup[param_name]:
        total += setup[param_name][key]
    print(total)

In [3]:
real_results_sql = f"""

select 
	fixture_id,
	max(team_id) filter (where team_id = team_home) as team_home,
	max(team_id) filter (where team_id = team_away) as team_away,
	max(goals_total) filter (where team_id = team_home) as home_goal,
	max(goals_total) filter (where team_id = team_away) as away_goal,
	max(shots_total) filter (where team_id = team_home) as home_shot,
	max(shots_total) filter (where team_id = team_away) as away_shot,
	max(passes_total) filter (where team_id = team_home) as home_passes,
	max(passes_total) filter (where team_id = team_away) as away_passes,
	max(dribbles_total) filter (where team_id = team_home) as home_dribble,
	max(dribbles_total) filter (where team_id = team_away) as away_dribble
	
from (
select
	fixture_id,
	team_id,
	sum(goals_total::int) as goals_total,
	sum(shots_total::int) as shots_total,
	sum(passes_accuracy::int) as passes_total,
	sum(tackles_total::int) as total_tackles,
	sum(interceptions::int) as interceptions,
	sum(blocks::int) as blocks,
	sum(dribbles_success::int) as dribbles_total
from ffl.fixture_player_performance
where fixture_id in (select id from ffl.fixtures where season = '{season}')
group by fixture_id, team_id) a
left join ffl.fixtures as fix on fix.id = fixture_id
group by 1
"""



In [4]:
fixtures_sql = f"""
select id, score_home, score_away from ffl.fixtures where season = '{season}'
"""
fixtures_list = []
cur.execute(fixtures_sql)
for fix in cur.fetchall():
    fixtures_list.append(fix[0])

In [5]:
def confusion_matrix(true_results=None, pred_results=None):
    pred_win_true_win = 0
    pred_win_true_loss = 0
    pred_win_true_tie = 0
    pred_loss_true_win = 0
    pred_loss_true_loss = 0
    pred_loss_true_tie = 0
    pred_tie_true_win = 0
    pred_tie_true_loss = 0
    pred_tie_true_tie = 0
    

    for row in range(len(true_results)):
        if pred_results.iloc[row]['home_goal'] > pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] > true_results.iloc[row]['away_goal']:
            pred_win_true_win += 1
        elif pred_results.iloc[row]['home_goal'] > pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] < true_results.iloc[row]['away_goal']:
            pred_win_true_loss += 1.
        elif pred_results.iloc[row]['home_goal'] > pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] == true_results.iloc[row]['away_goal']:
            pred_win_true_tie += 1
        elif pred_results.iloc[row]['home_goal'] < pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] > true_results.iloc[row]['away_goal']:
            pred_loss_true_win += 1
        elif pred_results.iloc[row]['home_goal'] < pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] < true_results.iloc[row]['away_goal']:
            pred_loss_true_loss += 1
        elif pred_results.iloc[row]['home_goal'] < pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] == true_results.iloc[row]['away_goal']:
            pred_loss_true_tie += 1
        elif pred_results.iloc[row]['home_goal'] == pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] > true_results.iloc[row]['away_goal']:
            pred_tie_true_win += 1
        elif pred_results.iloc[row]['home_goal'] == pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] < true_results.iloc[row]['away_goal']:
            pred_tie_true_loss += 1.
        elif pred_results.iloc[row]['home_goal'] == pred_results.iloc[row]['away_goal'] and true_results.iloc[row]['home_goal'] == true_results.iloc[row]['away_goal']:
            pred_tie_true_tie += 1
        

    confusion_matrix_df = pd.DataFrame({
        'labels':['True Win', 'True Loss', 'True_Tie'],
        'Pred Wins':[pred_win_true_win, pred_win_true_loss, pred_win_true_tie],
        'Pred Loss':[pred_loss_true_win, pred_loss_true_loss, pred_loss_true_tie],
        'Pred Ties':[pred_tie_true_win, pred_tie_true_loss, pred_tie_true_tie]
    })
    print(f'Total Accuracy:   {(pred_win_true_win + pred_loss_true_loss + pred_tie_true_tie) / len(true_results)}')
    print(f'Win Analysis:     Accuracy: {pred_win_true_win/(pred_win_true_win + pred_win_true_loss + pred_win_true_tie)}     ||     Recall: {pred_win_true_win / (pred_win_true_win + pred_loss_true_win + pred_tie_true_win)}')
    print(f'Loss Analysis:     Accuracy: {pred_loss_true_loss/(pred_loss_true_win + pred_loss_true_win + pred_loss_true_tie)}     ||     Recall: {pred_loss_true_loss / (pred_win_true_loss + pred_loss_true_loss + pred_tie_true_loss)}')
    if pred_tie_true_loss + pred_tie_true_loss + pred_tie_true_tie > 0:
        print(f'Tie Analysis:     Accuracy: {pred_tie_true_win/(pred_win_true_tie + pred_loss_true_tie + pred_tie_true_tie)}     ||     Recall: {pred_tie_true_win / (pred_tie_true_win + pred_tie_true_loss + pred_tie_true_tie)}')
    print(confusion_matrix_df.head())
        

In [6]:
simulation_iterations = 200

setup_json = {
    'team_home':None,
    'team_away':None,
    'minutes': 90,
    'ticks_per_minute': 15,
    'rolling_params' : {
        'keep': 0.24,
        'defence_initiation': 0.025,
        'pass': 0.695,
        'shoot':0.015,
        'dribble': 0.025
    },
    'home_advantage':0.006, #plus or minus this percent
    'mod_mode' : True,
    'shooting_accuracy_mod': -0.125, #changes performance by this amount
    'goalie_accuracy_mod': 0.275 #changes performance by this amount
}
check_roll_params(setup_json)

1.0


In [7]:
output_df = pd.DataFrame(columns=['fixture', 'team_home','team_away', 'home_goal', 'away_goal', 'home_shot', 'away_shot', 'home_pass', 'away_pass', 'home_dribble', 'away_dribble', ])
total_time = 0
for fix in fixtures_list:
    # time_start = time.time()
    output_df.loc[len(output_df)] = simulate(fix, setup_json, simulation_iterations)
    # total_time += time.time() - time_start

output_df = output_df.sort_values('fixture', ascending=True)
output_df = output_df.reset_index()
output_df = output_df.drop(['index'], axis=1)
real_results_df = pd.read_sql(real_results_sql, conn)
confusion_matrix(real_results_df, output_df)
# print(f'Fixture list iteration time {total_time}')

Total Accuracy:   0.36578947368421055
Win Analysis:     Accuracy: 0.5     ||     Recall: 0.41379310344827586
Loss Analysis:     Accuracy: 0.29411764705882354     ||     Recall: 0.2845528455284553
Tie Analysis:     Accuracy: 0.6265060240963856     ||     Recall: 0.3939393939393939
      labels  Pred Wins  Pred Loss  Pred Ties
0   True Win       72.0         50       52.0
1  True Loss       40.0         35       48.0
2   True_Tie       32.0         19       32.0


In [8]:
output_df.describe()

,fixture,team_home,team_away,home_goal,away_goal,home_shot,away_shot,home_pass,away_pass,home_dribble,away_dribble
count,3.800000e+02,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,1.035320e+06,112.600000,112.600000,1.547368,1.342105,10.055355,9.283197,394.877592,375.487158,9.284605,8.830816
std,1.655571e+02,286.479169,286.479169,0.818886,0.743532,2.501886,2.360173,64.399276,62.603865,1.408875,1.417134
min,1.035037e+06,33.000000,33.000000,0.000000,0.000000,4.705000,2.875000,239.115000,205.535000,5.330000,4.795000
25%,1.035132e+06,39.750000,39.750000,1.000000,1.000000,8.215000,7.561250,349.162500,330.098750,8.286250,7.886250
50%,1.035364e+06,47.500000,47.500000,1.000000,1.000000,9.755000,9.140000,390.375000,370.500000,9.287500,8.790000
75%,1.035458e+06,52.750000,52.750000,2.000000,2.000000,11.905000,10.791250,439.992500,416.775000,10.216250,9.706250
max,1.035553e+06,1359.000000,1359.000000,7.000000,5.000000,17.705000,17.300000,594.665000,549.245000,14.585000,12.790000


In [9]:
real_results_df.describe()

,fixture_id,team_home,team_away,home_goal,away_goal,home_shot,away_shot,home_passes,away_passes,home_dribble,away_dribble
count,3.800000e+02,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,1.035320e+06,112.600000,112.600000,1.728947,1.418421,10.952632,8.852632,405.431579,374.039474,8.907895,8.163158
std,1.655571e+02,286.479169,286.479169,1.326691,1.258712,4.575127,3.993975,153.828702,144.425350,3.968802,3.608069
min,1.035037e+06,33.000000,33.000000,0.000000,0.000000,1.000000,1.000000,100.000000,80.000000,1.000000,0.000000
25%,1.035132e+06,39.750000,39.750000,1.000000,0.000000,8.000000,6.000000,286.750000,265.000000,6.000000,5.000000
50%,1.035364e+06,47.500000,47.500000,2.000000,1.000000,10.000000,8.000000,393.500000,357.500000,8.000000,8.000000
75%,1.035458e+06,52.750000,52.750000,3.000000,2.000000,14.000000,11.000000,508.250000,463.000000,11.000000,10.000000
max,1.035553e+06,1359.000000,1359.000000,6.000000,8.000000,27.000000,22.000000,944.000000,827.000000,24.000000,22.000000
